# PRRS Virus Epidemiology & Pathogen Informatics Pipeline (Demo)
Author: Dr. Bernard Nwabueze Sunday (DVM, M.Sc)

This demo notebook generates a small synthetic PRRS dataset and runs a reproducible pipeline:
1. Data loading and cleaning
2. Exploratory Data Analysis (EDA)
3. Feature engineering
4. Train a baseline Random Forest classifier to predict outbreak
5. Save model and discuss next steps for genomic integration

Open in Colab: https://colab.research.google.com/github/Dr-Bernard/PRRS-Pipeline-Demo/blob/main/notebooks/PRRS_Pipeline_Demo.ipynb


## 1. Setup
Install required packages (Colab may already have most).

In [ ]:
# Install packages if running in Google Colab (uncomment if needed)
# !pip install scikit-learn pandas numpy matplotlib biopython joblib plotly

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import joblib
print("Libraries loaded. Pandas version:", pd.__version__)

## 2. Load synthetic dataset

In [ ]:
import os
data_path = "/mnt/data/data/synthetic_prrs_data.csv"
df = pd.read_csv(data_path)
print("Rows:", len(df))
df.head()

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Basic summaries
print(df.describe(include='all'))

# Outbreak counts by strain
strain_outbreak = df.groupby('Strain')['Outbreak'].mean().reset_index().sort_values('Outbreak', ascending=False)
print("\nOutbreak rate by Strain:")
print(strain_outbreak)

# Plot outbreak rate by strain (matplotlib)
plt.figure(figsize=(6,4))
plt.bar(strain_outbreak['Strain'], strain_outbreak['Outbreak'])
plt.title('Outbreak Rate by Strain')
plt.xlabel('Strain')
plt.ylabel('Outbreak rate')
plt.tight_layout()
plt.show()

## 4. Feature engineering

In [ ]:
# Simple feature engineering: encode categorical and create per-pig metrics
df2 = df.copy()
# One-hot encode strain
df2 = pd.get_dummies(df2, columns=['Strain'], drop_first=True)
# Per pig mortality
df2['Mortality_per_1000'] = df2['Mortality'] / df2['Pig_Count'] * 1000

features = ['Pig_Count','Temperature_C','Biosecurity_Score','Vaccination_Coverage','Antibody_Level',
            'Strain_PRRSV_B','Strain_PRRSV_C','Strain_PRRSV_D']
target = 'Outbreak'
X = df2[features].fillna(0)
y = df2[target]

X.head()

## 5. Modeling — Baseline Random Forest Classifier

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Train a Random Forest
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
print("Classification report:\n", classification_report(y_test, y_pred))

# Confusion matrix display
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test, y_pred), display_labels=model.classes_)
disp.plot()
plt.title('Confusion Matrix — Outbreak Prediction')
plt.show()

# Feature importances
importances = model.feature_importances_
sorted_idx = np.argsort(importances)[::-1]
for i in sorted_idx:
    print(f"{X.columns[i]}: {importances[i]:.4f}")

## 6. Save model and artifacts

In [ ]:
# Save the trained model and a sample of processed data
os.makedirs('/mnt/data/models', exist_ok=True)
joblib.dump(model, '/mnt/data/models/prrs_rf_model.pkl')
X_test_sample = X_test.copy()
X_test_sample['Outbreak_true'] = y_test.values
X_test_sample.to_csv('/mnt/data/data/processed_test_sample.csv', index=False)
print('Model and sample saved to /mnt/data/models and /mnt/data/data')

## 7. Phylogenetic & Sequence Analysis — Placeholder
This section shows where sequence-based analyses belong. For real PRRS analysis you'd:
1. Collect PRRS sequences (FASTA) and metadata
2. Align sequences with MAFFT
3. Build trees with IQ-TREE or RAxML
4. Run phylodynamic inference with BEAST

Below is an illustrative placeholder for loading a FASTA file (not included in this demo).

In [ ]:
# Placeholder code - uncomment and use real files in practice
# from Bio import SeqIO
# fasta_path = '/path/to/prrs_sequences.fasta'
# records = list(SeqIO.parse(fasta_path, 'fasta'))
# print('Number of sequences:', len(records))
# Example: write sequences to file or perform alignment with MAFFT (via subprocess)

## 8. Interactive Visualization (Plotly)
Simple interactive scatter of Antibody level vs Temperature colored by outbreak.

In [ ]:
import plotly.express as px
fig = px.scatter(df, x='Temperature_C', y='Antibody_Level', color=df['Outbreak'].astype(str),
                 hover_data=['Farm_ID','Strain','Pig_Count'], title='Antibody Level vs Temperature (Outbreak colored)')
fig.update_layout(height=600, width=900)
fig.show()

## 9. Discussion & Next Steps
- Replace synthetic data with real farm-level records and sequence FASTA files.
- Integrate MAFFT/IQ-TREE/BEAST steps in a separate pipeline (Snakemake/Docker recommended).
- Extend modeling to hierarchical models (GLMM) or phylodynamic-aware models to estimate strain-level effects.

### How to use this notebook
1. Upload your real `synthetic_prrs_data.csv` into `data/` or point to a cloud URL.
2. Add FASTA sequences and run alignment and tree-building steps.
3. Use model outputs as features in downstream analyses (e.g., outbreak risk dashboards).